# 暂时不用

In [59]:
import pandas as pd 
import numpy as np
# from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib
from sklearn.metrics import mean_squared_error as mse

data_train=pd.read_csv('data_train.txt',index_col=[0],header=0)
data_valid=pd.read_csv('data_valid.txt',index_col=[0],header=0)
# 完整的
data_train = data_train.append(data_valid)

features_list = list(data_train.columns)[3:7] + list(data_train.columns)[8:]
train_subset=data_train.loc[:,features_list]
valid_subset=data_valid.loc[:,features_list]
train_subset.head()

C:\Users\oyrx\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
C:\Users\oyrx\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,book,comment_max,comment_mean,comment_min,comments,emoji,english,forward_max,forward_mean,forward_min,...,tfidf,time,time_hour,time_weekday,time_weekend,title,user_id,video,vote,weibo_id
0,0,48.0,0.283186,0.0,0,0,0,114.0,0.830927,0.0,...,2.516793,2015-02-23 17:41:29,17,1,0,0,d38e9bed5d98110dc2489d0d1cac3c2a,0,0,7d45833d9865727a88b960b0603c19f6
1,0,48.0,0.283186,0.0,0,0,0,114.0,0.830927,0.0,...,2.473400,2015-03-14 21:22:57,21,6,1,0,d38e9bed5d98110dc2489d0d1cac3c2a,0,0,4fedf3888b1e16592f0e0bdc8b393845
2,0,48.0,0.283186,0.0,0,0,0,114.0,0.830927,0.0,...,1.328308,2015-06-18 16:25:36,16,4,0,0,d38e9bed5d98110dc2489d0d1cac3c2a,0,0,91be0b8612265aae32725cd4fa80b222
3,0,48.0,0.283186,0.0,0,0,0,114.0,0.830927,0.0,...,2.516793,2015-02-23 17:35:31,17,1,0,0,d38e9bed5d98110dc2489d0d1cac3c2a,0,0,bd2af99ecf1298f5539f0ddfcdd3ed64
4,0,48.0,0.283186,0.0,0,0,0,114.0,0.830927,0.0,...,2.516793,2015-02-23 17:30:13,17,1,0,0,d38e9bed5d98110dc2489d0d1cac3c2a,0,0,182078c5a409834f2128b3c9c2c289c3


In [2]:
def random_forest(data_train, data_test, estimator, depth, leaf_num, split_num, init=False):
    if init:
        data_test['repost_hat']=0
        data_test['comments_hat']=0
        data_test['likes_hat']=0
    else:
        pass
    
    #噪音数据预测值均为0,因此仅处理非噪音数据
    train=data_train[data_train['logit']==0]
    test=data_test[data_test['logit']==0]
    
    #分别对转发，评论和点赞建立三个森林
    forest_repost = RandomForestRegressor(n_estimators = estimator, oob_score = True, n_jobs = -1,random_state =50,
                                max_features = "auto", max_depth = depth, min_samples_leaf = leaf_num, min_samples_split = split_num) # criterion defalut by 'mse'
    forest_comments= RandomForestRegressor(n_estimators = estimator, oob_score = True, n_jobs = -1,random_state =50,
                                max_features = "auto", max_depth = depth, min_samples_leaf = leaf_num, min_samples_split = split_num)
    forest_likes = RandomForestRegressor(n_estimators = estimator, oob_score = True, n_jobs = -1,random_state =50,
                                max_features = "auto", max_depth = depth, min_samples_leaf = leaf_num, min_samples_split = split_num)
    #forest_repost = RandomForestRegressor(n_estimators = estimator, oob_score = True, n_jobs = -1
    #                            max_features = "auto") # criterion defalut by 'mse'
    #forest_comments= RandomForestRegressor(n_estimators = estimator, oob_score = True, n_jobs = -1,random_state =42,
    #                            max_features = "auto")
    #forest_likes = RandomForestRegressor(n_estimators = estimator, oob_score = True, n_jobs = -1,random_state =42,
    #                            max_features = "auto")
    
    #拟合三个森林
    regressor_train=train.drop(['repost','comments','likes','logit'],axis=1, inplace=False)
    repost_train=train.loc[:,['repost']]
    comments_train=train.loc[:,['comments']]
    likes_train=train.loc[:,['likes']]
    predict_repost=forest_repost.fit(regressor_train,repost_train.values.ravel()) # shape warning remove
    predict_comments=forest_comments.fit(regressor_train,comments_train.values.ravel())
    predict_likes=forest_likes.fit(regressor_train,likes_train.values.ravel())
    print(predict_repost.oob_score_, predict_comments.oob_score_, predict_likes.oob_score_)
    
    #预测测试集的数据
    regressor_test=test.drop(['repost','comments','likes','logit','repost_hat','comments_hat','likes_hat'],axis=1, inplace=False)
    repost_hat=np.round(predict_repost.predict(regressor_test),0)  #round函数只是返回四舍五入值，是浮点类型
    comments_hat=np.round(predict_comments.predict(regressor_test),0)
    likes_hat=np.round(predict_likes.predict(regressor_test),0)
    
    #将预测值赋值并设置为整数值
    data_test['repost_hat'][data_test['logit']==0]=repost_hat
    data_test['comments_hat'][data_test['logit']==0]=comments_hat
    data_test['likes_hat'][data_test['logit']==0]=likes_hat
    data_test['repost_hat']=data_test['repost_hat'].apply(lambda x:int(x))
    data_test['comments_hat']=data_test['comments_hat'].apply(lambda x:int(x))
    data_test['likes_hat']=data_test['likes_hat'].apply(lambda x:int(x))
    return data_test    


def precision(data):
    data['deviation_repost']=list(map(lambda x, y: abs(x-y)/(y+5), data['repost_hat'],data['repost']))
    #print (data['deviation_repost'])
    data['deviation_likes']=list(map(lambda x, y: abs(x-y)/(y+3), data['likes_hat'],data['likes']))
    #print (data['deviation_likes'])
    data['deviation_comments']=list(map(lambda x, y: abs(x-y)/(y+3), data['comments_hat'],data['comments']))
    #print (data['deviation_comments'])
    data['lcf_sum']=data['repost']+data['likes']+data['comments']
    #print (data['lcf_sum'])
    data['lcf_sum']=data['lcf_sum'].apply(lambda x: 100 if x>100 else x)
    data['precision_1_-0.8']=1-0.5*data['deviation_repost']-0.25*data['deviation_likes']-0.25*data['deviation_comments']-0.8
    #print (data['precision_1_-0.8'])
    data.loc[data['precision_1_-0.8']<=0,'sgn']=0
    data.loc[data['precision_1_-0.8']>0,'sgn']=1
    #print (data['sgn'])
    precision_=sum((data['lcf_sum']+1)*data['sgn'])/sum(data['lcf_sum']+1)
    return precision_

In [22]:
valid_subset=random_forest(train_subset, valid_subset, 500, 50, 10, 10) # estimator, depth, leaf_num, split_num, 
precision(valid_subset)

C:\Users\oyrx\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\oyrx\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\oyrx\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0.29030153173463363

# 训练随机森林模型

In [12]:
### constants
estimators = 1000
oob = True
random_num = 42
jobs = -1
depth = 50
leaf_num = 10
split_num = 50

def precision(data):
    data['deviation_repost']=list(map(lambda x, y: abs(x-y)/(y+5), data['repost_hat'],data['repost']))
    #print (data['deviation_repost'])
    data['deviation_likes']=list(map(lambda x, y: abs(x-y)/(y+3), data['likes_hat'],data['likes']))
    #print (data['deviation_likes'])
    data['deviation_comments']=list(map(lambda x, y: abs(x-y)/(y+3), data['comments_hat'],data['comments']))
    #print (data['deviation_comments'])
    data['lcf_sum']=data['repost']+data['likes']+data['comments']
    #print (data['lcf_sum'])
    data['lcf_sum']=data['lcf_sum'].apply(lambda x: 100 if x>100 else x)
    data['precision_1_-0.8']=1-0.5*data['deviation_repost']-0.25*data['deviation_likes']-0.25*data['deviation_comments']-0.8
    #print (data['precision_1_-0.8'])
    data.loc[data['precision_1_-0.8']<=0,'sgn']=0
    data.loc[data['precision_1_-0.8']>0,'sgn']=1
    #print (data['sgn'])
    precision_=sum((data['lcf_sum']+1)*data['sgn'])/sum(data['lcf_sum']+1)
    return precision_


x=data_train[features_list]
y=data_train['repost']
model_repost =  RandomForestRegressor( n_estimators = estimators , oob_score = oob, random_state = random_num,  n_jobs = jobs,
                                max_features = "auto", max_depth = depth, min_samples_leaf = leaf_num, min_samples_split = split_num)
model_repost.fit(x,y)
#model_repost.score

x=data_train[features_list]
y=data_train['comments']
model_comments =  RandomForestRegressor( n_estimators = estimators , oob_score = oob, random_state = random_num,  n_jobs = jobs,
                                max_features = "auto", max_depth = depth, min_samples_leaf = leaf_num, min_samples_split = split_num)
model_comments.fit(x,y)
#model_comments.score

x=data_train[features_list]
y=data_train['likes']
model_likes =  RandomForestRegressor( n_estimators = estimators , oob_score = oob, random_state = random_num,  n_jobs = jobs,
                                max_features = "auto", max_depth = depth, min_samples_leaf = leaf_num, min_samples_split = split_num)
model_likes.fit(x,y)
#model_likes.score

model_repost.oob_score_, model_comments.oob_score_, model_likes.oob_score_

(0.14790829919259862, 0.15063227028339232, 0.37990754777761826)

# Load Pre-trained Random Forest Model

In [82]:
### LOAD THE MODEL FROM THE DISK
model_repost = joblib.load('./model/model_repost.sav')
model_comments = joblib.load('./model/model_comments.sav')
model_likes = joblib.load('./model/model_likes.sav')

# 随机森林、逻辑回归、均值复合预测

In [156]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor

data_train=pd.read_csv('data_train.txt',index_col=[0],header=0)
data_valid=pd.read_csv('data_valid.txt',index_col=[0],header=0)
#data_train=data_train.append(data_valid)

features_list = ['tfidf', 'time_weekday', 'time_weekend', 'time_hour', 'panduan',
                   'length_all', 'length_chinese', 'english', 'non_ch', 'sharing', 'auto',
                   'interaction', 'book', 'mention', 'vote', 'lottery', 'emoji', 'video',
                   'http', 'stock', 'app', 'title', 'ad', 'hotwords', 'keywords',
                   'is_noise', 'number_in_train', 'forward_max', 'comment_max', 'like_max',
                   'forward_min', 'comment_min', 'like_min', 'forward_mean',
                   'comment_mean', 'like_mean', 'forward_more_ave_pr',
                   'comment_more_ave_pr', 'like_more_ave_pr', 'max_f/l', 'max_c/l',
                   'min_f/l', 'min_c/l', 'mean_f/l', 'mean_c/l', 'night', 'logit']
train_subset=data_train.loc[:,features_list]
valid_subset=data_valid.loc[:,features_list]

noise_cancellation = True


C:\Users\oyrx\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [157]:
data_valid_test = data_valid
data_valid_test['repost_hat']   =0
data_valid_test['comments_hat'] =0
data_valid_test['likes_hat']    =0
if noise_cancellation == True:
    #噪音数据预测值均为0,因此仅处理非噪音数据[['logit']==0]
    x = data_valid_test[data_valid_test['logit'] == 0][features_list]
    data_valid_test[data_valid_test['logit'] == 0]['repost_hat'],   \
    data_valid_test[data_valid_test['logit'] == 0]['comments_hat'], \
    data_valid_test[data_valid_test['logit'] == 0]['likes_hat']     \
                                =                                   \
                                model_repost.predict(x),            \
                                model_comments.predict(x),          \
                                model_likes.predict(x)

else:
    x = data_valid_test[features_list]
    data_valid_test['repost_hat'],   \
    data_valid_test['comments_hat'], \
    data_valid_test['likes_hat']     \
                                =                                   \
                                model_repost.predict(x),            \
                                model_comments.predict(x),          \
                                model_likes.predict(x)
precision(data_valid_test[['repost', 'comments', 'likes', 'repost_hat', 'comments_hat', 'likes_hat']])

C:\Users\oyrx\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\oyrx\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\oyrx\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

0.2408603143197938

In [158]:
tmp = data_valid_test[['user_id', 'repost', 'comments', 'likes', 'repost_hat', 'comments_hat', 'likes_hat']]

m = data_train.groupby(['user_id'])[['repost', 'comments', 'likes']].median()
m = m.sort_values(by='repost', ascending=False)
m.columns = ['repost_hat', 'comments_hat', 'likes_hat']
tmp = pd.merge(tmp, m, how='left', on='user_id')

tmp['repost_hat_y'] = tmp['repost_hat_y'].fillna(tmp['repost_hat_x'])
tmp['comments_hat_y'] = tmp['comments_hat_y'].fillna(tmp['comments_hat_x'])
tmp['likes_hat_y'] = tmp['likes_hat_y'].fillna(tmp['likes_hat_x'])
tmp['repost_hat'], tmp['comments_hat'] , tmp['likes_hat']  =  tmp['repost_hat_y'] ,tmp['comments_hat_y'],tmp['likes_hat_y'] 
precision(tmp[['repost', 'comments', 'likes', 'repost_hat', 'comments_hat', 'likes_hat']])


C:\Users\oyrx\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\oyrx\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\oyrx\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

0.3003861595223833

### MSE on valid set

In [159]:
#mse(data_valid['repost'], data_valid['repost_hat'].apply(lambda x: int(x*1.3)))
print(mse(tmp['repost'], tmp['repost_hat']),\
         mse(tmp['comments'], tmp['comments_hat']), \
         mse(tmp['likes'], tmp['likes_hat']))

7739.445409957983 129.67003050799613 757.1965567220733


### 寻找最佳的整数转换系数

In [132]:
int_tmp = tmp.loc[:, ['repost', 'comments', 'likes', 'repost_hat', 'comments_hat', 'likes_hat']]
i = 1.0
best = [0, 0]
for j in range(1, 100):
    for col in ['repost_hat', 'comments_hat', 'likes_hat']:
        int_tmp.loc[:, col] = int_tmp.loc[:, col].apply(lambda x:int(i * x))
    score = precision(int_tmp)
    #print(i, score)
    if score > best[1]:
        best[0], best[1] = i, score
    else:
        pass
    i += 0.01
print(best)

[1.09, 0.31350886929008964]


### 复合模型

In [147]:
#读取和定义数据
data_predict=pd.read_csv('data_predict_tfidf.txt',index_col=[0],header=0)
x = data_predict[features_list]

data_predict['repost_hat'],         \
data_predict['comments_hat'],       \
data_predict['likes_hat']           \
        =                           \
        model_repost.predict(x),    \
        model_comments.predict(x),  \
        model_likes.predict(x)

tmp = data_predict[['user_id', 'weibo_id', 'repost_hat', 'comments_hat', 'likes_hat']]
m = data_train.groupby(['user_id'])[['repost', 'comments', 'likes']].median()
m = m.sort_values(by='repost', ascending=False)
m.columns = ['repost_hat', 'comments_hat', 'likes_hat']
tmp = pd.merge(tmp, m, how='left', on='user_id')

tmp['repost_hat_y']   = tmp['repost_hat_y'].fillna(tmp['repost_hat_x'])
tmp['comments_hat_y'] = tmp['comments_hat_y'].fillna(tmp['comments_hat_x'])
tmp['likes_hat_y']    = tmp['likes_hat_y'].fillna(tmp['likes_hat_x'])
tmp['repost_hat'], tmp['comments_hat'] , tmp['likes_hat']  = tmp['repost_hat_y'] ,tmp['comments_hat_y'],tmp['likes_hat_y'] 

if noise_cancellation == True:
    tmp[tmp['logit'] == 0]['repost_hat'], tmp[tmp['logit'] == 0]['comments_hat'] , tmp[tmp['logit'] == 0]['likes_hat']  = 0, 0, 0

result = tmp[['user_id', 'weibo_id', 'repost_hat', 'comments_hat', 'likes_hat']]

for col in ['repost_hat', 'comments_hat', 'likes_hat']:
    result[col] = result[col].apply(lambda x:int(1.3 * x)) # best mutiplier
result.head()

C:\Users\oyrx\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,user_id,weibo_id,repost_hat,comments_hat,likes_hat
0,fa5aed172c062c61e196eac61038a03b,7cce78a4ad39a91ec1f595bcc7fb5eba,0,0,0
1,77fc723c196a45203e70f4d359c96946,a3494d8cf475a92739a2ffd421640ddf,1,0,0
2,e4097b07f34366399b623b94f174f60c,6b89aea5aa7af093dde0894156c49dd3,0,0,0
3,d43f7557c303b84070b13aa4eeeb21d3,0bdeff19392e15737775abab46dc5437,0,0,0
4,87465974e53e9f047e355e6e9b135b55,545c14094cbe50679daa63fe16419111,0,0,0


In [148]:
 # 按照线上提交要求重组并存储数据
result = result.assign(
    predict=result.repost_hat.astype(str) + ',' +
    result.comments_hat.astype(str)       + ',' +
    result.likes_hat.astype(str))         # longer but faster than apply lambda
result = result[['user_id', 'weibo_id', 'predict']]
result.columns = ['uid', 'mid', 'predict']
result.to_csv("./result.txt", header=False, sep="\t", index=False)

# Save the model

In [29]:
### SAVE THE MODEL TO DISK
joblib.dump(model_repost, './model/model_repost.sav')
joblib.dump(model_repost, './model/model_comments.sav')
joblib.dump(model_repost, './model/model_likes.sav')